<a href="https://colab.research.google.com/github/shivendrra/SmallLanguageModel-project/blob/main/Final%20Model/Colab%20Notebooks/GPTfromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run this first always!
!pip install python-dotenv
!pip install youtube-transcript-api

In [ ]:
channel_Id_Json  = [
  "UCb_MAhL8Thb3HJ_wPkH3gcw", #phil edwards
  "UCA295QVkf9O1RQ8_-s3FVXg", #aevy tv
  "UCpFFItkfZz1qz5PpHpqzYBw", #nexpo
  "UCY1kMZp36IQSyNx_9h4mpCg", #mark robber
  "UCA19mAJURyYHbJzhfpqhpCA", #action lab shorts
  "UCqnbDFdCpuN8CMEg0VuEBqA", #new york times
  "UCddiUEpeqJcYeBxX1IVBKvQ", #the verge
  "UCcefcZRL2oaA_uBNeo5UOWg", #y-combinator
  "UCLXo7UDZvByw2ixzpQCufnA", #vox
  "UCsQoiOrh7jzKmE8NBofhTnQ", #varun mayya
  "UCUyvQV2JsICeLZP4c_h40kA", #thomas flight
  "UCvjgXvBlbQiydffZU7m1_aw", #the coding train
  "UCRI00CwLZdLRCWg5BdDOsNw", #canadian lad
  "UCEIwxahdLz7bap-VDs9h35A", #steve mould
  "UC4bq21IPPbpu0Qrsl7LW0sw", #slidebean
  "UCR1IuLEqb6UEA_zQ81kwXfg", #real engineering
  "UCIlU5KDHKFSaebYviKfOidw", #newsthink
  "UCtYKe7-XbaDjpUwcU5x0bLg", #neo
  "UCBJycsmduvYEL83R_U4JriQ", #mkbdh
  "UCRcgy6GzDeccI7dkbbBna3Q", #lemmino
  "UC3_BakzLfadvFrsnClMFWmQ", #john coogan
  "UCmGSJVG3mCRXVOP4yZrU1Dw", #johnny harris
  "UCFN6lQpfY8XIRdhv9G-f4bg", #henry belcaster
  "UConJDkGk921yT9hISzFqpzw", #freethink
  "UClWTCPVi-AU9TeCN6FkGARg", #EO
  "UCyHJ94JzwY92NsBVzJ2aE3Q", #econ
  "UCTqEu1wZDBju2tHkNP1dwzQ", #earthrise
  "UCcabW7890RKJzL968QWEykA", #CS 50
  "UCamLstJyCa-t5gfZegxsFMw", #colin and samir
  "UC415bOPUcGSamy543abLmRA", #cleo abraham
  "UCpMcsdZf2KkAnfmxiq2MfMQ", #arvin ash
  "UCqVEHtQoXHmUCfJ-9smpTSg", #answer in progress
  "UCYO_jab_esuFRV4b17AJtAw", #3blue1brown
  "UCHnyfMqiRRG1u-2MsSQLbXA", #veritasium
  "UCsXVk37bltHxD1rDPwtNM8Q", #kurzgesagt
  "UC9RM-iSvTu1uPJb8X5yp3EQ" #wendover
]

In [ ]:
import json
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('yt_secret_key')

In [ ]:
from googleapiclient.discovery import build
from youtube_transcript_api import TranscriptsDisabled, YouTubeTranscriptApi
import logging

logging.basicConfig(filename='youtube_fetch.log', level=logging.ERROR)
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
import timeit

start_time = timeit.default_timer()

videoNo = 0
for links in channel_Id_Json:
  next_page_token = None
  videoIds = []

  while True:
    channelRes = youtube.channels().list(
      part='contentDetails', id=links
    ).execute()

    if 'items' in channelRes and channelRes['items']:
      playlistId = channelRes['items'][0]['contentDetails']['relatedPlaylists']['uploads']

      playlistResult = youtube.playlistItems().list(
        part='contentDetails', playlistId=playlistId,
        maxResults = 100, pageToken = next_page_token
      ).execute()

      videoIds.extend([item['contentDetails']['videoId'] for item in playlistResult.get('items', [])])

      next_page_token = playlistResult.get('nextPageToken')

      if not next_page_token:
        break

  for ids in videoIds:
    videoUrl = f"https://www.youtube.com/watch?v={ids}"
    try:
      raw_transcripts = []
      try:
        captions = YouTubeTranscriptApi.get_transcript(
          ids, languages=['en'], preserve_formatting=True
        )
        if captions:
          formatted_captions = [{'text': caption['text']} for caption in captions]
          raw_transcripts.append(formatted_captions)
          videoNo += 1
          print(f"Number of videos with valid captions are: {videoNo}")
        else:
          continue
      except TranscriptsDisabled as e:
        print(F"There was an error while getting the captions: {e}")
      except Exception as e:
        logging.error(f"There was some error while fetching the video: {str(e)}")
    except Exception as e:
      logging.error(f"There was some error while getting the captions: {str(e)}")

    with open('training_data.txt', 'a', encoding='utf-8') as file:
      for videoCaptions in raw_transcripts:
        for line in videoCaptions:
          file.write(line['text'] + ' ')

print(f"time taken to execute the code is {(timeit.default_timer() - start_time) / 60} mins")

**Coding the GPT from here**

In [ ]:
with open('training_data.txt', 'r', encoding='utf-8') as file:
  text = file.read()

In [ ]:
# cleaning the text data
import re
text = re.sub('[^a-zA-Z0-9\s.,!?]', '', text)

In [ ]:
# total no of chars and vocab size
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
# enoder - decoder
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
# tokenizing the data + train-test split
import torch

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]